In [2]:
# 데이터셋 로드
# 자전거 대여 데이터 로드
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


그레이디언트 부스팅
  - 앙상블 학습방법중에 하나
  - 약한 예측모형을 결합해서 강력한 예측 모형을 만들어냄
  - 이전 오차를 보완
    - 손실함수를 사용한다.

In [3]:
import xgboost as xgb
xgb.set_config(verbosity = 0)

In [4]:
df_bikes = pd.read_csv('https://raw.githubusercontent.com/leekyuyoung20231211/python/main/data/bike_rentals_cleaned.csv')
df_bikes.head(1)

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985


In [5]:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bikes,y_bikes,random_state=2)

그레이디언트 부스팅 모델 만들기

In [12]:
from sklearn.tree import DecisionTreeRegressor
tree_1 =  DecisionTreeRegressor(max_depth=2, random_state=2)
tree_1.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [13]:
# 예측
y_train_pred = tree_1.predict(X_train)

In [15]:
# 잔차를 계산
y2_train = y_train - y_train_pred

# tree를 초기화
tree_2 = DecisionTreeRegressor(max_depth=2,random_state=2)
# 잔차에 모델을 훈련
tree_2.fit(X_train,y2_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [16]:
# 예측
y2_train_pred = tree_2.predict(X_train)
#잔차를 계산
y3_train = y2_train - y2_train_pred
# tree 초기화
tree_3 = DecisionTreeRegressor(max_depth=2,random_state=2)
tree_3.fit(X_train,y3_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [21]:
y_pred =  tree_1.predict(X_test) + tree_2.predict(X_test) + tree_3.predict(X_test)
from sklearn.metrics import mean_squared_error as mse
result = mse(y_test,y_pred)  # 평균 제곱오차
# 평균 제곱근 오차
result**0.5

911.0479538776444

In [22]:
# 평균 제곱근 오차
mse(y_test,y_pred,squared=False)

911.0479538776444

사이킷런의 그레이디언트 부스팅

In [26]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3,random_state=2,learning_rate=1.0)
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)
# 평균제곱오차
mse(y_test,y_pred,squared=False), mse(y_test,y_pred)**0.5

(911.0479538776439, 911.0479538776439)

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=30,random_state=2,learning_rate=1.0)
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)
# 평균제곱오차
mse(y_test,y_pred,squared=False), mse(y_test,y_pred)**0.5

(857.1072323426944, 857.1072323426944)

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300,random_state=2,learning_rate=1.0)
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)
# 평균제곱오차
mse(y_test,y_pred,squared=False), mse(y_test,y_pred)**0.5

(936.3617413678853, 936.3617413678853)

In [36]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300,random_state=2)
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)
# 평균제곱오차
mse(y_test,y_pred,squared=False), mse(y_test,y_pred)**0.5

(653.7456840231495, 653.7456840231495)

그레이디언트 부스팅 튜닝

In [41]:
learning_rate_values = {
    'learning_rate' :[0.001,0.01,0.05,0.1,0.15,0.2,0.3,0.5,1.0]
    , 'n_estimators':[300,500,100]
    , 'max_depth':[2,3]
    }
from sklearn.model_selection import GridSearchCV
model = GradientBoostingRegressor(max_depth=2, n_estimators=300,random_state=2)
gridsearch = GridSearchCV(model,param_grid=learning_rate_values,cv=5,scoring='neg_mean_squared_error')
gridsearch.fit(X_bikes,y_bikes)
print(f"best param : {gridsearch.best_params_}")
best_model = gridsearch.best_estimator_
y_pred = best_model.predict(X_test)
print(f"rmse : {mse(y_test,y_pred)**0.5}")

best param : {'learning_rate': 0.3, 'max_depth': 2, 'n_estimators': 300}
rmse : 222.87215115217546


대회참가

In [42]:
!unzip '/content/drive/MyDrive/데이콘/웹로그기반조회수예측.zip'
import pandas as pd
df = pd.read_csv('/content/open/train.csv')
X = df.loc[:,'browser':]
y = df['TARGET']

test_df = pd.read_csv('/content/open/test.csv')
test_df = test_df[ X.columns]

# 전처리 - 임이의 빈 값으로 전처리
temp = X.isna().mean()
for na_obj_col in temp[temp>0].index:
  X[na_obj_col] = X[na_obj_col].fillna('na')

temp = test_df.isna().mean()
for na_obj_col in temp[temp>0].index:
  test_df[na_obj_col] = test_df[na_obj_col].fillna('na')

# 2.전처리 - object를 수치형으로 변경
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
object_colunms = X.select_dtypes(include=['object']).columns

# 학습용데이터를 기준으로 라벨인코딩을하는데 검증용데이터중에 추가되는 범주형 데이터가 있으면 라벨인코더에 추가한다
import numpy as np
for colname in object_colunms:
  le.fit(X[colname])
  X[colname] = le.transform(X[colname])

  for case_ in np.unique(test_df[colname]):
    if case_ not in le.classes_:
      le.classes_ = np.append(le.classes_,case_)

  test_df[colname]  =  le.transform(test_df[colname])

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X)
X_scaled = ss.transform(X)
X_test_scaled = ss.transform(test_df)


Archive:  /content/drive/MyDrive/데이콘/웹로그기반조회수예측.zip
   creating: open/
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           
  inflating: open/train.csv          


그레디언트 부스팅
  - 하이퍼 파라메터 튜닝

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=2)
learning_rate_values = {
    'learning_rate' :[0.001,0.01,0.05,0.1,0.15,0.2,0.3,0.5,1.0]
    , 'n_estimators':[300,500,100]
    , 'max_depth':[2,3]
    }
from sklearn.model_selection import GridSearchCV
model = GradientBoostingRegressor(max_depth=2, n_estimators=300,random_state=2)
gridsearch = GridSearchCV(model,param_grid=learning_rate_values,cv=5,scoring='neg_mean_squared_error',n_jobs = -1)
gridsearch.fit(X_train,y_train)
print(f"best param : {gridsearch.best_params_}")
best_model = gridsearch.best_estimator_
y_pred = best_model.predict(X_test)
print(f"rmse : {mse(y_test,y_pred)**0.5}")

In [ ]:
predict = best_model.predict(X_test_scaled)
submission_df = pd.read_csv('/content/open/sample_submission.csv')
submission_df['TARGET'] = predict
submission_df.to_csv("제출.csv",index=False)

외계행성 데이터 찾기

In [ ]:
!unzip "/content/drive/MyDrive/data/exoplanets.csv.zip" -d "./exoplanets"

In [ ]:
filepath = ""
df= df.read_csv(filepath)
df.head(1)

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sum()

In [ ]:
# 데이터 나누기
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train,X_test,y_train,y_test =  train_test_split(X,y,random_state = 2)

그레이디언트 분류모델 생성

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
# 정확도 평가
from sklearn.metrics import accuracy_score

시간측정

In [ ]:
import time
start = time.time()
df.info()
end = time.time()
print(f"실행시간 : {end-start}")

속도 비교

In [ ]:
start = time.time()
gbc = GradientBoostingClassifier(n_estimators=100, max_depth=2,random_state=2)
gbc.fit(X_train,y_train)
y_pred = gbc.predict(X_test)
score = accuracy_score(y_test,y_pred)
print(f"점수 : {score}")
end = time.time()
print(f"실행시간 : {end-start}")

In [ ]:
start = time.time()
xgbc = XGBClassifier(n_estimators=100, max_depth=2,random_state=2)
xgbc.fit(X_train,y_train)
y_pred = xgbc.predict(X_test)
score = accuracy_score(y_test,y_pred)
print(f"점수 : {score}")
end = time.time()
print(f"실행시간 : {end-start}")